# Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

## Variables

In [2]:
N_FEATURES_LIST = [1, 3, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 90, 94]

MUTUAL_INFORMATION = "MUTUAL_INFORMATION"
SPEARMAN = "SPEARMAN"
ONE_R = "ONE_R"

FEATURE_SELECTION_LIST = [MUTUAL_INFORMATION, SPEARMAN, ONE_R]

CLASSIFIERS = "Classifiers"
ACCURACY = "Accuracy"
N_FEATURES_NAME = "N_Features"
FEATURE_SELECTION = "Feature_Selection"
DIVERSITY = "Diversity"
FUSION_CLASSIFIER = "Fusion"


### Variables file

In [3]:
REAL_RETURN = "Real Return"
RISK = "Risk"

UNIQUE_CLASSIFIERS = "######## CLASSIFICADORES UNICOS ########"
CV = "######## CV ########"
SELECTION_CLASSIFIERS = "######## CLASSIFIERS SELECTION ########"
WHITOUT_DIVERSITY = "######## WHITOUT DIVERSITY ########"
BAGGING = "######## BAGGING ########"
ADABOOST = "######## ADABOOST ########"
FUSION = "######## FUSION ########"
END = "END"

RANDOM_FOREST = "RANDOM FOREST"
SVM = "SVM"
DECISION_TREE = "DECISION TREE"
NAIVE_BAYES = "NAIVE BAYES"
NEURAL_NETWORK = "NEURAL NETWORK"
LOGISTIC_REGRESSION = "LOGISTIC REGRESSION"
KNEIGHBORS= "KNEIGHBORS"
XGBOOST = "XGBOOST"

CLASSIFIER_LIST = [RANDOM_FOREST, SVM, DECISION_TREE, NAIVE_BAYES, NEURAL_NETWORK, LOGISTIC_REGRESSION, KNEIGHBORS, XGBOOST]

## Create Dataframes

In [4]:
UNIQUE_COLUMNS = [N_FEATURES_NAME, FEATURE_SELECTION, CLASSIFIERS, ACCURACY]
DF_UNIQUE_REAL_RETURN = pd.DataFrame(columns=UNIQUE_COLUMNS)
DF_UNIQUE_RISK = pd.DataFrame(columns=UNIQUE_COLUMNS)

FUSION_COLUMNS = [N_FEATURES_NAME, DIVERSITY, FUSION_CLASSIFIER, ACCURACY]
DF_FUSION_REAL_RETURN = pd.DataFrame(columns=FUSION_COLUMNS)
DF_FUSION_RISK = pd.DataFrame(columns=FUSION_COLUMNS)

## Read File

In [5]:
def addUniqueClassifiers(n_features:int, method:str, unique_classifiers:pd.DataFrame):
  global DF_UNIQUE_REAL_RETURN
  global DF_UNIQUE_RISK
  
  for key in unique_classifiers.keys():
    row_result = [n_features, method, key, unique_classifiers[key][0]]
    row_risk = [n_features, method, key, unique_classifiers[key][1]]

    temp_df = pd.DataFrame([row_result], columns=DF_UNIQUE_REAL_RETURN.columns)
    DF_UNIQUE_REAL_RETURN = pd.concat([DF_UNIQUE_REAL_RETURN, temp_df], ignore_index=True)

    temp_df = pd.DataFrame([row_risk], columns=DF_UNIQUE_RISK.columns)
    DF_UNIQUE_RISK = pd.concat([DF_UNIQUE_RISK, temp_df], ignore_index=True)

In [6]:
def addFusion(n_features:int, fusion_list:list, df_base:pd.DataFrame):
  for result in fusion_list:
    row = [n_features, 
           str(result[0]).replace("[", "").replace("]", ""),  
           str(result[1]).replace("[", "").replace("]", ""),
           result[2]]
    
    temp_df = pd.DataFrame([row], columns=df_base.columns)
    df_base = pd.concat([df_base, temp_df], ignore_index=True)

  return df_base

In [7]:
def readFile(selection_feature:str, n_features:int, type:str):
  global DF_FUSION_REAL_RETURN
  global DF_FUSION_RISK

  FILE_PATH = f"./files/{selection_feature}_{n_features}_FUSION_{type}.txt"
  file = open(FILE_PATH, 'r')

  unique_classifiers = {}
  cv_classifiers = {}

  real_return_whiout = []
  risk_whiout = []

  real_return_bagging = []
  risk_bagging = []

  real_return_adaboost = []
  risk_adaboost = []

  real_return_fusion = []
  risk_fusion = []

  for line in file:
    if UNIQUE_CLASSIFIERS in line:
      line = file.readline()
      while CV not in line:
        line = line.strip()
        classifier = line.split(":")
        if classifier.__len__() > 1:
          unique_classifiers[classifier[0]] = eval(classifier[1])
        line = file.readline()
    if CV in line:
      line = file.readline()
      while SELECTION_CLASSIFIERS not in line:
        line = line.strip()
        line = file.readline()
    if WHITOUT_DIVERSITY in line:
      line = file.readline()
      while BAGGING not in line:
        line = line.strip()
        bg = line.split(": ")
        if bg.__len__() > 1 and bg[0] == REAL_RETURN:
          real_return_whiout.append(bg[1].split(', '))
        if bg.__len__() > 1 and bg[0] == RISK:
          risk_whiout.append(bg[1].split(', '))
        line = file.readline()
    if BAGGING in line:
      line = file.readline()
      while ADABOOST not in line:
        line = line.strip()
        bg = line.split(": ")
        if bg.__len__() > 1 and bg[0] == REAL_RETURN:
          real_return_bagging.append(bg[1].split(', '))
        if bg.__len__() > 1 and bg[0] == RISK:
          risk_bagging.append(bg[1].split(', '))
        line = file.readline()
    if ADABOOST in line:
      line = file.readline()
      while FUSION not in line:
        line = line.strip()
        bg = line.split(": ")
        if bg.__len__() > 1 and bg[0] == REAL_RETURN:
          real_return_adaboost.append(bg[1].split(', '))
        if bg.__len__() > 1 and bg[0] == RISK:
          risk_adaboost.append(bg[1].split(', '))
        line = file.readline()
    if FUSION in line:
      line = file.readline()
      while END not in line:
        line = line.strip().replace(":", "").split(" ")
        if line.__len__() > 1 and line[2] == "RealReturn":
          real_return_fusion.append((line[0], line[1], line[3]))
        if line.__len__() > 1 and line[2] == "Risk":
          risk_fusion.append((line[0], line[1], line[3]))
        line = file.readline()

  addUniqueClassifiers(n_features, selection_feature, unique_classifiers)
  DF_FUSION_REAL_RETURN = addFusion(n_features, real_return_fusion, DF_FUSION_REAL_RETURN)
  DF_FUSION_RISK = addFusion(n_features, risk_fusion, DF_FUSION_RISK)

In [8]:
def queryByColumn(df:pd.DataFrame, column:str, value:str):
  return df[df[column] == value]

In [9]:
readFile(SPEARMAN, 80, "RISK")

In [10]:
def plot_accuracy(df, type):
  # Cria um gráfico para cada classificador
  for classifier in CLASSIFIER_LIST:
    # Filtre o dataframe pelo classificador atual
    df_classifier = df[df[CLASSIFIERS] == classifier]

    # Crie um novo gráfico
    plt.figure()

    # Plote as linhas para cada FEATURE_SELECTION
    for feature_selection in df_classifier[FEATURE_SELECTION].unique():
      df_selection = df_classifier[df_classifier[FEATURE_SELECTION] == feature_selection]
      plt.plot(df_selection[N_FEATURES_NAME], df_selection[ACCURACY], label=feature_selection)

    # Configure os eixos X e Y
    plt.xlabel('Número de Features')
    plt.ylabel('Acurácia')

    # Configure o título do gráfico
    title = f'{type} - Classificador: {classifier}'
    plt.title(title)

    # Adicione uma legenda
    plt.legend()

    # Adicione um grid
    plt.grid(color='gray', linestyle=':', linewidth=0.5)
    plt.xticks(range(0, df[N_FEATURES_NAME].max()+1, 5))
    plt.savefig(f'./images/FUSION_{type}_{classifier}.png')

  # Exiba todos os gráficos
  plt.show()


### Real Return

In [11]:
queryByColumn(DF_FUSION_REAL_RETURN, DIVERSITY, "WHITOUT")

,N_Features,Diversity,Fusion,Accuracy
0,80,WHITOUT,RANDOM_FOREST,0.9394099051633298
3,80,WHITOUT,SVM,0.940858798735511
6,80,WHITOUT,DECISION_TREE,0.9392781875658588
9,80,WHITOUT,NAIVE_BAYES,0.9077976817702845
12,80,WHITOUT,NEURAL_NETWORK,0.940595363540569
15,80,WHITOUT,LOGISTIC_REGRESSION,0.940858798735511
18,80,WHITOUT,KNeighborsClassifier,0.9351949420442571
21,80,WHITOUT,XG_BOOST,0.9395416227608009


In [12]:
queryByColumn(DF_FUSION_REAL_RETURN, DIVERSITY, "BAGGING")

,N_Features,Diversity,Fusion,Accuracy
1,80,BAGGING,RANDOM_FOREST,0.9336143308746049
4,80,BAGGING,SVM,0.9338777660695469
7,80,BAGGING,DECISION_TREE,0.9333508956796628
10,80,BAGGING,NAIVE_BAYES,0.8938356164383562
13,80,BAGGING,NEURAL_NETWORK,0.9338777660695469
16,80,BAGGING,LOGISTIC_REGRESSION,0.9338777660695469
19,80,BAGGING,KNeighborsClassifier,0.9328240252897787
22,80,BAGGING,XG_BOOST,0.9334826132771338


In [13]:
queryByColumn(DF_FUSION_REAL_RETURN, DIVERSITY, "ADABOOST")

,N_Features,Diversity,Fusion,Accuracy
2,80,ADABOOST,RANDOM_FOREST,0.9392781875658588
5,80,ADABOOST,SVM,0.9392781875658588
8,80,ADABOOST,DECISION_TREE,0.9392781875658588
11,80,ADABOOST,NAIVE_BAYES,0.9371707060063225
14,80,ADABOOST,NEURAL_NETWORK,0.9386195995785037
17,80,ADABOOST,LOGISTIC_REGRESSION,0.9371707060063225
20,80,ADABOOST,KNeighborsClassifier,0.9329557428872497
23,80,ADABOOST,XG_BOOST,0.9392781875658588


In [14]:
DF_FUSION_REAL_RETURN.sort_values(ACCURACY, ascending=False)

,N_Features,Diversity,Fusion,Accuracy
15,80,WHITOUT,LOGISTIC_REGRESSION,0.940858798735511
3,80,WHITOUT,SVM,0.940858798735511
12,80,WHITOUT,NEURAL_NETWORK,0.940595363540569
21,80,WHITOUT,XG_BOOST,0.9395416227608009
0,80,WHITOUT,RANDOM_FOREST,0.9394099051633298
5,80,ADABOOST,SVM,0.9392781875658588
6,80,WHITOUT,DECISION_TREE,0.9392781875658588
2,80,ADABOOST,RANDOM_FOREST,0.9392781875658588
23,80,ADABOOST,XG_BOOST,0.9392781875658588
8,80,ADABOOST,DECISION_TREE,0.9392781875658588


### Risk

In [15]:
queryByColumn(DF_FUSION_RISK, DIVERSITY, "WHITOUT")

,N_Features,Diversity,Fusion,Accuracy
0,80,WHITOUT,RANDOM_FOREST,0.7868809272918862
3,80,WHITOUT,SVM,0.7888566912539515
6,80,WHITOUT,DECISION_TREE,0.7870126448893572
9,80,WHITOUT,NAIVE_BAYES,0.7469704952581665
12,80,WHITOUT,NEURAL_NETWORK,0.7879346680716544
15,80,WHITOUT,LOGISTIC_REGRESSION,0.77963645943098
18,80,WHITOUT,KNeighborsClassifier,0.7111433087460485
21,80,WHITOUT,XG_BOOST,0.7871443624868283


In [16]:
queryByColumn(DF_FUSION_RISK, DIVERSITY, "BAGGING")

,N_Features,Diversity,Fusion,Accuracy
1,80,BAGGING,RANDOM_FOREST,0.7925447839831401
4,80,BAGGING,SVM,0.7925447839831401
7,80,BAGGING,DECISION_TREE,0.791095890410959
10,80,BAGGING,NAIVE_BAYES,0.7289251844046365
13,80,BAGGING,NEURAL_NETWORK,0.7925447839831401
16,80,BAGGING,LOGISTIC_REGRESSION,0.7926765015806112
19,80,BAGGING,KNeighborsClassifier,0.7648840885142255
22,80,BAGGING,XG_BOOST,0.7925447839831401


In [17]:
queryByColumn(DF_FUSION_RISK, DIVERSITY, "ADABOOST")

,N_Features,Diversity,Fusion,Accuracy
2,80,ADABOOST,RANDOM_FOREST,0.8419388830347735
5,80,ADABOOST,SVM,0.8419388830347735
8,80,ADABOOST,DECISION_TREE,0.8419388830347735
11,80,ADABOOST,NAIVE_BAYES,0.8385142255005269
14,80,ADABOOST,NEURAL_NETWORK,0.8419388830347735
17,80,ADABOOST,LOGISTIC_REGRESSION,0.8419388830347735
20,80,ADABOOST,KNeighborsClassifier,0.8069020021074815
23,80,ADABOOST,XG_BOOST,0.8419388830347735


In [18]:
DF_FUSION_RISK.sort_values(ACCURACY, ascending=False)

,N_Features,Diversity,Fusion,Accuracy
23,80,ADABOOST,XG_BOOST,0.8419388830347735
2,80,ADABOOST,RANDOM_FOREST,0.8419388830347735
5,80,ADABOOST,SVM,0.8419388830347735
8,80,ADABOOST,DECISION_TREE,0.8419388830347735
17,80,ADABOOST,LOGISTIC_REGRESSION,0.8419388830347735
14,80,ADABOOST,NEURAL_NETWORK,0.8419388830347735
11,80,ADABOOST,NAIVE_BAYES,0.8385142255005269
20,80,ADABOOST,KNeighborsClassifier,0.8069020021074815
16,80,BAGGING,LOGISTIC_REGRESSION,0.7926765015806112
4,80,BAGGING,SVM,0.7925447839831401
